In [1]:
# PTZ Central Control
# All connected cameras are controlled thorugh this script
from reolink_aio.api import Host
import asyncio
import numpy as np

In [2]:
# Global Variable
pan_speed = [5, 5, 5]
angular_error_tolerance = 20 # ptz_pan_pos index
angular_scale = 0.225 # Each increment on ptz_pan_pos = 0.225 (360/)

In [3]:
# PTZ Camera Angular Waypoints (Yaw)
ptzcam01_angular_waypoints = [688, 977, 196]
ptzcam02_angular_waypoints = [1592, 393, 1238]
ptzcam03_angular_waypoints = [1289, 946, 1498]

In [4]:
# Create a host-object for 3 cameras
n_cam = 3
print('Awakening the Mahcine Spirit')
print('May the Omnissiah bless this sacred machine')
print('...')
ptzcam01 = Host('192.168.123.252', 'admin', 'tobeor!2b', port=443) #PTZCAM01
ptzcam02 = Host('192.168.123.61', 'admin', 'tobeor!2b', port=443) # PTZCAM02
ptzcam03 = Host('192.168.123.221', 'admin', 'tobeor!2b', port=443) # PTZCAM03
ptzcam = {}
ptzcam['0'] = ptzcam01
ptzcam['1'] = ptzcam02
ptzcam['2'] = ptzcam03

print('Performing the Ritual of Access')
print('...')
print('Augur Array Online: ')
for i in range(n_cam):
    current_camera = ptzcam[str(i)]
    await current_camera.get_host_data()
    current_camera_name = current_camera.nvr_name
    print('   '+str(current_camera_name))

Awakening the Mahcine Spirit
May the Omnissiah bless this sacred machine
...
Performing the Ritual of Access
...
Augur Array Online: 
   PTZCAM01
   PTZCAM02
   PTZCAM03


In [6]:
# Rotate all camera toward its origin
print('Initiate Ritual of Reset')
print('...')

n_cam01_stop = True
n_cam02_stop = True
n_cam03_stop = True

while True:
    # Log current pan angle of each auguar array
    await ptzcam01.get_host_data()
    cam01_pan = ptzcam01.ptz_pan_position(channel=0)
    await ptzcam02.get_host_data()
    cam02_pan = ptzcam02.ptz_pan_position(channel=0)
    await ptzcam03.get_host_data()
    cam03_pan = ptzcam03.ptz_pan_position(channel=0)

    pan_speed_rest = [1, 10]
    print('Camera 01: ' + str(np.abs(cam01_pan-ptzcam01_angular_waypoints[0])) + 
          ' | Camera 02: ' + str(np.abs(cam02_pan-ptzcam02_angular_waypoints[0])) + 
          ' | Camera 03: ' + str(np.abs(cam03_pan-ptzcam03_angular_waypoints[0])))
    
    # Cam01
    if n_cam01_stop:
        await ptzcam01.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed_rest[0], patrol=None)
        if np.abs(cam01_pan-ptzcam01_angular_waypoints[0]) <= angular_error_tolerance:
            await ptzcam01.set_ptz_command(channel=0, command='Stop', preset=None, speed = pan_speed_rest[0], patrol=None)
            n_cam01_stop = False
            print('Camera 01 resetted')
        elif np.abs(cam01_pan-ptzcam01_angular_waypoints[0]) >= 250:
            await ptzcam01.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed_rest[1], patrol=None)

    # Cam02
    if n_cam02_stop:
        await ptzcam02.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed_rest[0], patrol=None)
        if np.abs(cam02_pan - ptzcam02_angular_waypoints[0]) <= angular_error_tolerance:
            await ptzcam02.set_ptz_command(channel=0, command='Stop', preset=None, speed = pan_speed_rest[0], patrol=None)
            n_cam02_stop = False
            print('Camera 02 resetted')
        elif np.abs(cam02_pan - ptzcam02_angular_waypoints[0]) >= 250:
            await ptzcam02.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed_rest[1], patrol=None)

    # Cam03
    if n_cam03_stop:
        await ptzcam03.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed_rest[0], patrol=None)
        if np.abs(cam03_pan - ptzcam03_angular_waypoints[0]) <= angular_error_tolerance:
            await ptzcam03.set_ptz_command(channel=0, command='Stop', preset=None, speed = pan_speed_rest[0], patrol=None)
            n_cam03_stop = False
            print('Camera 03 resetted')
        elif np.abs(cam03_pan - ptzcam03_angular_waypoints[0]) >= 250:
            await ptzcam03.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed_rest[1], patrol=None)

    if n_cam01_stop + n_cam02_stop + n_cam03_stop == 0:
        print('By the blessing of the Omnissiah, cameras are resetted')
        break

Initiate Ritual of Reset
...
Camera 01: 401 | Camera 02: 270 | Camera 03: 149
Camera 01: 328 | Camera 02: 207 | Camera 03: 119
Camera 01: 242 | Camera 02: 153 | Camera 03: 102
Camera 01: 190 | Camera 02: 137 | Camera 03: 87
Camera 01: 176 | Camera 02: 123 | Camera 03: 73
Camera 01: 160 | Camera 02: 108 | Camera 03: 58
Camera 01: 145 | Camera 02: 93 | Camera 03: 43
Camera 01: 132 | Camera 02: 79 | Camera 03: 30
Camera 01: 118 | Camera 02: 65 | Camera 03: 15
Camera 03 resetted
Camera 01: 103 | Camera 02: 50 | Camera 03: 7
Camera 01: 90 | Camera 02: 37 | Camera 03: 7
Camera 01: 77 | Camera 02: 24 | Camera 03: 7
Camera 01: 64 | Camera 02: 11 | Camera 03: 7
Camera 02 resetted
Camera 01: 51 | Camera 02: 2 | Camera 03: 7
Camera 01: 41 | Camera 02: 2 | Camera 03: 7
Camera 01: 30 | Camera 02: 2 | Camera 03: 7
Camera 01: 20 | Camera 02: 2 | Camera 03: 7
Camera 01 resetted
By the blessing of the Omnissiah, cameras are resetted


In [5]:
# Control all three camera at once
print('Initiate Ritual of Surveillance')
print('May the Omnissiah grants this machine its blessing')
print('...')
cam01_waypoint = 1
cam02_waypoint = 1
cam03_waypoint = 1
while True:
    # Log current pan angle of each auguar array
    await ptzcam01.get_host_data()
    cam01_pan = ptzcam01.ptz_pan_position(channel=0)
    await ptzcam02.get_host_data()
    cam02_pan = ptzcam02.ptz_pan_position(channel=0)
    await ptzcam03.get_host_data()
    cam03_pan = ptzcam03.ptz_pan_position(channel=0)

    # Send command toward first waypoint
    print('Camera 01: ' + str(np.abs(cam01_pan-ptzcam01_angular_waypoints[cam01_waypoint])) + 
          ' | Camera 02: ' + str(np.abs(cam02_pan-ptzcam02_angular_waypoints[cam02_waypoint])) + 
          ' | Camera 03: ' + str(np.abs(cam03_pan-ptzcam03_angular_waypoints[cam03_waypoint])))
    
    # Cam01
    if cam01_waypoint == 1:
        await ptzcam01.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[0], patrol=None)
        # print('Camera 01: ' + str(np.abs(cam01_pan-ptzcam01_angular_waypoints[cam01_waypoint])))
        if np.abs(cam01_pan - ptzcam01_angular_waypoints[cam01_waypoint]) <= angular_error_tolerance:
            await ptzcam01.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[0], patrol=None)
            cam01_waypoint += 1
            # print('Camera 01 Waypoint '+ str(cam01_waypoint)+' Reached')
    else:
        await ptzcam01.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[0], patrol=None)
        # print('Camera 01: ' + str(np.abs(cam01_pan-ptzcam01_angular_waypoints[cam01_waypoint])))
        if np.abs(cam01_pan - ptzcam01_angular_waypoints[cam01_waypoint]) <= angular_error_tolerance:
            await ptzcam01.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[0], patrol=None)
            cam01_waypoint -= 1
            # print('Camera 01 Waypoint '+ str(cam01_waypoint)+' Reached')
    
    # # Cam01
    # if cam01_waypoint == 1:
    #     await ptzcam01.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[0], patrol=None)
    #     # print('Camera 01: ' + str(np.abs(cam01_pan-ptzcam01_angular_waypoints[cam01_waypoint])))
    #     if np.abs(cam01_pan - ptzcam01_angular_waypoints[cam01_waypoint]) <= angular_error_tolerance:
    #         await ptzcam01.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[0], patrol=None)
    #         cam01_waypoint += 1
    #         # print('Camera 01 Waypoint '+ str(cam01_waypoint)+' Reached')
    # else:
    #     await ptzcam01.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[0], patrol=None)
    #     # print('Camera 01: ' + str(np.abs(cam01_pan-ptzcam01_angular_waypoints[cam01_waypoint])))
    #     if np.abs(cam01_pan - ptzcam01_angular_waypoints[cam01_waypoint]) <= angular_error_tolerance:
    #         await ptzcam01.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[0], patrol=None)
    #         cam01_waypoint -= 1
    #         # print('Camera 01 Waypoint '+ str(cam01_waypoint)+' Reached')
    
    # Cam02
    if cam02_waypoint == 1:
        await ptzcam02.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[1], patrol=None)
        # print('Camera 02: ' + str(np.abs(cam02_pan-ptzcam02_angular_waypoints[cam02_waypoint])))
        if np.abs(cam02_pan - ptzcam02_angular_waypoints[cam02_waypoint]) <= angular_error_tolerance:
            await ptzcam02.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[1], patrol=None)
            cam02_waypoint += 1
            # print('Camera 02 Waypoint '+ str(cam02_waypoint)+' Reached')
    else:
        await ptzcam02.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[1], patrol=None)
        # print('Camera 02: ' + str(np.abs(cam02_pan-ptzcam02_angular_waypoints[cam02_waypoint])))
        if np.abs(cam02_pan - ptzcam02_angular_waypoints[cam02_waypoint]) <= angular_error_tolerance:
            await ptzcam02.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[1], patrol=None)
            cam02_waypoint -= 1
            # print('Camera 02 Waypoint '+ str(cam02_waypoint)+' Reached')

    # # Cam02
    # if cam02_waypoint == 1:
    #     await ptzcam02.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[1], patrol=None)
    #     # print('Camera 02: ' + str(np.abs(cam02_pan-ptzcam02_angular_waypoints[cam02_waypoint])))
    #     if np.abs(cam02_pan - ptzcam02_angular_waypoints[cam02_waypoint]) <= angular_error_tolerance:
    #         await ptzcam02.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[1], patrol=None)
    #         cam02_waypoint += 1
    #         # print('Camera 02 Waypoint '+ str(cam02_waypoint)+' Reached')
    # else:
    #     await ptzcam02.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[1], patrol=None)
    #     # print('Camera 02: ' + str(np.abs(cam02_pan-ptzcam02_angular_waypoints[cam02_waypoint])))
    #     if np.abs(cam02_pan - ptzcam02_angular_waypoints[cam02_waypoint]) <= angular_error_tolerance:
    #         await ptzcam02.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[1], patrol=None)
    #         cam02_waypoint -= 1
    #         # print('Camera 02 Waypoint '+ str(cam02_waypoint)+' Reached')


    # Cam03
    if cam03_waypoint == 1:
        await ptzcam03.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[1], patrol=None)
        # print('Camera 03: ' + str(np.abs(cam03_pan-ptzcam03_angular_waypoints[cam03_waypoint])))
        if np.abs(cam03_pan - ptzcam03_angular_waypoints[cam03_waypoint]) <= angular_error_tolerance:
            await ptzcam03.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[1], patrol=None)
            cam03_waypoint += 1
            # print('Camera 03 Waypoint '+ str(cam03_waypoint)+' Reached')
    else:
        await ptzcam03.set_ptz_command(channel=0, command='Right', preset=None, speed = pan_speed[1], patrol=None)
        # print('Camera 03: ' + str(np.abs(cam03_pan-ptzcam03_angular_waypoints[cam03_waypoint])))
        if np.abs(cam03_pan - ptzcam03_angular_waypoints[cam03_waypoint]) <= angular_error_tolerance:
            await ptzcam03.set_ptz_command(channel=0, command='Left', preset=None, speed = pan_speed[1], patrol=None)
            cam03_waypoint -= 1
            # print('Camera 03 Waypoint '+ str(cam03_waypoint)+' Reached')

Initiate Ritual of Surveillance
May the Omnissiah grants this machine its blessing
...
Camera 01: 288 | Camera 02: 392 | Camera 03: 357
Camera 01: 276 | Camera 02: 378 | Camera 03: 341
Camera 01: 250 | Camera 02: 350 | Camera 03: 313
Camera 01: 219 | Camera 02: 320 | Camera 03: 283
Camera 01: 191 | Camera 02: 292 | Camera 03: 255
Camera 01: 164 | Camera 02: 263 | Camera 03: 226
Camera 01: 133 | Camera 02: 234 | Camera 03: 197
Camera 01: 106 | Camera 02: 206 | Camera 03: 169
Camera 01: 77 | Camera 02: 177 | Camera 03: 140
Camera 01: 49 | Camera 02: 149 | Camera 03: 113
Camera 01: 22 | Camera 02: 122 | Camera 03: 85
Camera 01: 7 | Camera 02: 92 | Camera 03: 56
Camera 01: 795 | Camera 02: 59 | Camera 03: 22
Camera 01: 766 | Camera 02: 32 | Camera 03: 4
Camera 01: 736 | Camera 02: 2 | Camera 03: 550
Camera 01: 702 | Camera 02: 843 | Camera 03: 517
Camera 01: 673 | Camera 02: 871 | Camera 03: 490
Camera 01: 647 | Camera 02: 899 | Camera 03: 462
Camera 01: 618 | Camera 02: 926 | Camera 03: 4

CancelledError: 

In [15]:
await ptzcam01.set_ptz_command(channel=0, command="Stop", preset=None, speed=10, patrol=None) # Control PTZ
await ptzcam02.set_ptz_command(channel=0, command="Stop", preset=None, speed=10, patrol=None) # Control PTZ
await ptzcam03.set_ptz_command(channel=0, command="Stop", preset=None, speed=10, patrol=None) # Control PTZ